In [13]:
import os
import requests
import datetime
import numpy as np
import xarray
import rasterio as rio
import rioxarray

from shapely import Polygon

import geopandas as gpd
import pandas as pd

In [14]:
###
#output code to a misc file if testing (0 for no, 1 for yes)
isTest = 0

site='ross1'
shpDate='_2018-10-08'
shpCycle='_01'
shpPath = f"../shapes/{site}/{site}{shpDate}{shpCycle}.geojson"

# start cycle must be chosen to match polygon date
cycStart = 1
cycEnd = 22

# save flag (0: dont save, 1: save)
saveFlag=1
filename=f'../shapes/{site}/{site}_{cycStart:02}_{cycEnd:02}'
filepath=f'../shapes/{site}/'

if isTest==1: 
    filename=f'../shapes/testFolder/{site}_{cycStart:02}_{cycEnd:02}'
    filepath=f'../shapes/testFolder/'

###

In [15]:
#lookup tables
global cycleDict
#cycle start and end dates
cycleDict = {'01': ['2018-10-13', '2018-12-28'], '02': ['2018-12-28', '2019-03-29'], 
             '03': ['2019-03-29', '2019-06-28'], '04': ['2019-07-09', '2019-09-26'], 
             '05': ['2019-09-26', '2019-12-26'], '06': ['2019-12-26', '2020-03-26'],
             '07': ['2020-03-26', '2020-06-25'], '08': ['2020-06-25', '2020-09-24'],
             '09': ['2020-09-24', '2020-12-23'], '10': ['2020-12-24', '2021-03-24'],
             '11': ['2021-03-24', '2021-06-23'], '12': ['2021-06-23', '2021-09-22'],
             '13': ['2021-09-22', '2021-12-22'], '14': ['2021-12-22', '2022-03-23'],
             '15': ['2022-03-23', '2022-06-21'], '16': ['2022-06-21', '2022-09-20'],
             '17': ['2022-09-20', '2022-12-20'], '18': ['2022-12-20', '2023-03-21'],
             '19': ['2023-03-21', '2023-06-20'], '20': ['2023-06-20', '2023-09-19'],
             '21': ['2023-09-19', '2023-12-18'], '22': ['2023-12-18', '2024-03-18']}

def advectPoint(date1, date2, x, y, vxr):
    v = vxr.sel(x=x, y=y, method='nearest')
    dateformat = '%Y-%m-%d'
    t1 = datetime.datetime.strptime(date1, dateformat)
    t2 = datetime.datetime.strptime(date2, dateformat)
    dt = (t2-t1)
    dt = dt.days/365
    #print(dt)
    xp, yp = x+(v.vx*dt), y+(v.vy*dt)
    #print(v.vx.values[0])
    #print(dt)
    #print(f'{xp.values[0]}, {yp.values[0]}')
    return xp.values[0], yp.values[0]

def advectPoly(gdf, cycA, cycB, vxr):
    currentCycle = str(f'{cycA:02}')
    nextCycle = str(f'{cycB:02}')
    date1 = cycleDict[currentCycle][0]
    date2 = cycleDict[nextCycle][0]
    #print(f'date1 {date1}')
    #print(f'date2 {date2}')
    df = list(gdf.geometry.exterior.iloc[0].coords)
    #print(df)
    x = df[0][0]
    y = df[0][1]
    outList = []
    #print(cycA)
    for i in range(len(df)):
        x = df[i][0]
        y = df[i][1]
        #print(f'x, y in : {x}, {y}')
        xp, yp = advectPoint(date1, date2, x, y, vxr)
        outList.append((xp, yp))
        #print(f'x, y out : {xp}, {yp}')

    outGDF = gpd.GeoDataFrame({'cycle': cycB, 'geometry': [Polygon(outList)]}).set_index('cycle')
    outGDF.crs = 'EPSG:3031'
    #advectedPoly = outGDF.to_crs('EPSG:4326')
    
    return outGDF
    

In [16]:
# Load polygon
print('loading polygon')
shpFile = gpd.read_file(shpPath)
shpFile.crs = 'EPSG:3031'
poly = list(shpFile.geometry.exterior[0].coords)
poly4326 = list(shpFile.to_crs('EPSG:4326').geometry.exterior[0].coords)
polyDict = {
    'EPSG:3031' : poly,
    'EPSG:4326' : poly4326
}  
# Package into a dataframe
print('packaging into dataframe')
df = pd.DataFrame(polyDict, columns=['EPSG:3031', 'EPSG:4326'])

#Load velocity data
print('loading velocity data')
velocity_data_fn = '../data/itslive/ANT_G0240_0000.nc'
vxr = rioxarray.open_rasterio(velocity_data_fn)

## Advect polgyon
print('advecting polygon')
allCycles = [cycStart]
allPolys = [Polygon(df['EPSG:4326'])]
currentPoly = gpd.GeoDataFrame({'cycle': cycStart, 'geometry': [Polygon(df['EPSG:3031'])]}).set_index('cycle')
currentPoly.crs = 'EPSG:3031'
if saveFlag==1:
        currentGDF = gpd.GeoDataFrame({'cycle': cycStart, 
            'geometry': currentPoly.to_crs('EPSG:4326').geometry}).set_index('cycle').set_geometry('geometry')
        currentPoly.to_file(f"{filepath}{site}_{cycStart:02}.geojson",
            driver='GeoJSON')
#print(currentPoly)

print('Advecting cycle...')
for i in range(cycStart, cycEnd):
    print(f'{i}')
    currentCycle = str(f'{i:02}')
    currentDate = cycleDict[currentCycle]
    nextCycle = str(f'{i+1:02}')
    nextDate = cycleDict[nextCycle]
    currentPoly = advectPoly(currentPoly, i, i+1, vxr)
    #print(currentPoly)
    currentPoly.crs = 'EPSG:3031'

    allCycles.append(i+1)
    allPolys.append(currentPoly.to_crs('EPSG:4326').geometry.iloc[0])
    # save
    if saveFlag==1:
        currentGDF = gpd.GeoDataFrame({'cycle': i+1, 
            'geometry': currentPoly.to_crs('EPSG:4326').geometry}).set_index('cycle').set_geometry('geometry')
        currentPoly.to_file(f"{filepath}{site}_{nextCycle}.geojson",
            driver='GeoJSON')
        #print(currentPoly.to_crs('EPSG:4326'))

polyGDF = gpd.GeoDataFrame({'cycle': allCycles, 'geometry': allPolys}).set_index('cycle')

if saveFlag==1:
    print(f'Saving file as geojson named {filename}.geojson')
    polyGDF.to_file(f"{filename}.geojson", driver='GeoJSON')

loading polygon
packaging into dataframe
loading velocity data
advecting polygon
Advecting cycle...
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
Saving file as geojson named ../shapes/ross1/ross1_01_22.geojson


In [17]:
polyGDF

,geometry
cycle,
1,"POLYGON ((-164.78441 -78.91087, -167.11194 -78..."
2,"POLYGON ((-164.78449 -78.91061, -167.11123 -78..."
3,"POLYGON ((-164.78457 -78.91029, -167.11037 -78..."
4,"POLYGON ((-164.78464 -78.90992, -167.10941 -78..."
5,"POLYGON ((-164.78470 -78.90963, -167.10865 -78..."
6,"POLYGON ((-164.78477 -78.90929, -167.10777 -78..."
7,"POLYGON ((-164.78483 -78.90896, -167.10689 -78..."
8,"POLYGON ((-164.78490 -78.90863, -167.10600 -78..."
9,"POLYGON ((-164.78496 -78.90830, -167.10511 -78..."
